In [4]:

import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

seed = 42


def make_image_df(root_dir, extensions=('jpg', 'png', 'jpeg')):
    recs = []
    for label in sorted(os.listdir(root_dir)):
        d = os.path.join(root_dir, label)
        if not os.path.isdir(d): continue
        for fn in os.listdir(d):
            if fn.lower().endswith(extensions):
                recs.append({'filepath': os.path.join(d, fn),
                             'label': label})
    return pd.DataFrame(recs)


# 1) Build DataFrames & splits
df_tv = make_image_df('data/asl_alphabet')
df_test = make_image_df('data/synthetic_test')
df_train, df_dev = train_test_split(
    df_tv, stratify=df_tv['label'],
    test_size=0.2, random_state=seed
)

# 2) Encode labels
le = LabelEncoder().fit(df_tv['label'])
y_train = le.transform(df_train['label'])
y_dev = le.transform(df_dev['label'])
y_test = le.transform(df_test['label'])


# 3) Loader: grey → resize → normalize [0,1] → flatten
def load_flatten_norm(paths, size=(64, 64)):
    arrs = []
    for p in paths:
        img = Image.open(p).convert('L').resize(size)
        a = np.asarray(img, np.float32) / 255.0
        arrs.append(a.ravel())
    return np.vstack(arrs)


train_files = df_train['filepath'].tolist()
dev_files = df_dev['filepath'].tolist()
test_files = df_test['filepath'].tolist()

X_train = load_flatten_norm(train_files)
X_dev = load_flatten_norm(dev_files)
X_test = load_flatten_norm(test_files)

# 4) PCA + StandardScaler (fixed n_components)
pca_components = 20
pca = PCA(n_components=pca_components, random_state=seed)
X_train_pca = pca.fit_transform(X_train)
X_dev_pca = pca.transform(X_dev)
X_test_pca = pca.transform(X_test)

scaler = StandardScaler().fit(X_train_pca)
X_train_s = scaler.transform(X_train_pca)
X_dev_s = scaler.transform(X_dev_pca)
X_test_s = scaler.transform(X_test_pca)


In [5]:
import os
from itertools import product
from concurrent.futures import ThreadPoolExecutor, as_completed

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report

seed = 42

# 1) Define model specs (no more PCA or scaler here)
model_specs = {
    "RandomForest": {
        "cls": RandomForestClassifier,
        "init_kwargs": {"random_state": seed},
        "param_grid": {
            "n_estimators": [100, 200, 300],
            "max_depth": [10, 20],
            "max_features": [0.05, 0.1, 0.2],
            "min_samples_split": [2, 4],
        }
    },
}

# 2) Expand tasks
tasks = []
for name, spec in model_specs.items():
    keys, values = zip(*spec["param_grid"].items())
    for combo in product(*values):
        params = dict(zip(keys, combo))
        tasks.append((name, spec["cls"], spec["init_kwargs"], params))


# 3) Worker: fit on TRAIN PCA+scaled, eval on DEV PCA+scaled
def worker(name, Cls, init_kwargs, params):
    est = Cls(**init_kwargs)
    est.set_params(**params)
    est.fit(X_train_s, y_train)
    preds = est.predict(X_dev_s)
    return name, params, f1_score(y_dev, preds, average="macro"), f1_score(y_test, est.predict(X_test_s),
                                                                           average="macro")


# 4) Run in parallel
print(f"Running {len(tasks)} tasks…")
results = []
with ThreadPoolExecutor(max_workers=os.cpu_count() - 1 or 1) as exe:
    futures = [exe.submit(worker, *t) for t in tasks]
    for i, fut in enumerate(as_completed(futures), 1):
        name, params, f1_dev, f1_test = fut.result()
        print(f"{i / len(tasks):.2f} -- {name:20s} {params} → Dev F1 = {f1_dev:.4f}, Test F1 = {f1_test:.4f}")
        results.append((name, params, f1_dev, f1_test))

# 5) Pick best per model & retrain on TRAIN only
best_estimators = {}
for name, spec in model_specs.items():
    subset = [(f1_dev, f1_test, params) for m, params, f1_dev, f1_test in results if m == name]
    best_f1_dev, best_f1_test, best_params = max(subset, key=lambda x: x[0])
    print(f"→ Best {name}: F1={best_f1_dev:.4f} with {best_params} \t Test F1 = {best_f1_test:.4f}")

    final = spec["cls"](**spec["init_kwargs"])
    final.set_params(**best_params)
    final.fit(X_train_s, y_train)
    best_estimators[name] = final

Running 36 tasks…
0.03 -- RandomForest         {'n_estimators': 100, 'max_depth': 10, 'max_features': 0.05, 'min_samples_split': 2} → Dev F1 = 0.8176, Test F1 = 0.0226
0.06 -- RandomForest         {'n_estimators': 100, 'max_depth': 10, 'max_features': 0.05, 'min_samples_split': 4} → Dev F1 = 0.8106, Test F1 = 0.0254
0.08 -- RandomForest         {'n_estimators': 100, 'max_depth': 20, 'max_features': 0.05, 'min_samples_split': 4} → Dev F1 = 0.9880, Test F1 = 0.0245
0.11 -- RandomForest         {'n_estimators': 100, 'max_depth': 20, 'max_features': 0.05, 'min_samples_split': 2} → Dev F1 = 0.9905, Test F1 = 0.0269
0.14 -- RandomForest         {'n_estimators': 200, 'max_depth': 10, 'max_features': 0.05, 'min_samples_split': 2} → Dev F1 = 0.8274, Test F1 = 0.0273
0.17 -- RandomForest         {'n_estimators': 100, 'max_depth': 10, 'max_features': 0.1, 'min_samples_split': 4} → Dev F1 = 0.8010, Test F1 = 0.0240
0.19 -- RandomForest         {'n_estimators': 100, 'max_depth': 10, 'max_features':

KeyboardInterrupt: 

In [10]:
all_labels = list(range(len(le.classes_)))
all_names = list(le.classes_)


def evaluate(est, Xs, y, split):
    preds = est.predict(Xs)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds, average="macro")
    print(f"\n=== {est.__class__.__name__} on {split} ===")
    print(f"{split} Accuracy: {acc * 100:.2f}%, Macro-F1: {f1 * 100:.2f}%")
    print(classification_report(
        y, preds,
        labels=all_labels,
        target_names=all_names,
        zero_division=0,
        digits=4
    ))


# 7) Report on DEV & TEST
for name, est in best_estimators.items():
    evaluate(est, X_dev_s, y_dev, "Dev")
    evaluate(est, X_test_s, y_test, "Test")